# Implementing Flat Tax Rate
We will be looking at what people actually pay, what they should pay, and how to decrease to a flat tax rate. \
Evan Sellers + Michael Yager

In [1]:
import pandas as pd
import seaborn as sns

In [2]:
df = pd.read_csv("./data/tax_data_2020.csv")
df = df[df.zipcode != 0]

In [3]:
def toMillion(amount):
    return round(amount / 1000000, 2)

def toBillion(amount):
    return round(amount / 1000000000, 2)

def toTrillion(amount):
    return round(amount / 1000000000000, 10)

## Percentage of Income Tax - Paided

In [4]:
# Remove Rows w/ Zero Income
df = df[df["A02650"] != 0]

In [5]:
df["taxPercentagePaid"] = df.apply(lambda row: row["A06500"] / row["A00100"], axis=1)

In [6]:
paid = df.groupby(["agi_stub"])["taxPercentagePaid"].mean()
paid

agi_stub
1    0.012746
2    0.044214
3    0.065613
4    0.078532
5    0.110051
6    0.188869
Name: taxPercentagePaid, dtype: float64

## Percentage of Income Tax - Expected

### Returns

In [7]:
df["taxPercentageReturn"] = df.apply(lambda row: 1 - (row["A00100"] / row["A02650"]), axis=1)

In [8]:
returns = df.groupby(["agi_stub"])["taxPercentageReturn"].mean()
returns

agi_stub
1    0.019301
2    0.012092
3    0.011414
4    0.009370
5    0.010816
6    0.010638
Name: taxPercentageReturn, dtype: float64

### Credits

In [9]:
TAX_CREDIT_AMT_1040  = [ "A07225", "A11070", "A10960", "A10970" ]
TAX_CREDIT_AMT_SCH3  = [ "A07230", "A07240", "A07180", "A07300", "A07260", "A09400", "A11450", "A11560" ]

In [10]:
def sumColumns(dataframe, columns):
    total = 0
    for col in columns:
        total += abs(dataframe[col])
    return total

In [11]:
df["credits"] = df.apply(lambda row: (sumColumns(row, TAX_CREDIT_AMT_SCH3) + sumColumns(row, TAX_CREDIT_AMT_1040)), axis=1)

In [12]:
df["taxPercentageCredit"] = df.apply(lambda row: row["credits"] / row["A02650"], axis=1)

In [13]:
df.groupby(["agi_stub"])["credits"].sum()

agi_stub
1    46161900.0
2    54919370.0
3    34827480.0
4    25014840.0
5    52657448.0
6    41222786.0
Name: credits, dtype: float64

In [14]:
credits = df.groupby(["agi_stub"])["taxPercentageCredit"].mean()
credits

agi_stub
1    0.065316
2    0.036898
3    0.025105
4    0.019558
5    0.014639
6    0.005525
Name: taxPercentageCredit, dtype: float64

This is intresting. This means lower income brackets take advantage of tax credits, and they are a larger impact to their overall taxes.

### Total

In [15]:
expected = paid + returns + credits
expected

agi_stub
1    0.097362
2    0.093203
3    0.102132
4    0.107461
5    0.135507
6    0.205032
dtype: float64

## Flat Tax Rate Basic
According to the congress report we have to account for `$1,609 Billion` in revenue for income taxes. \
[Revenues in Fiscal Year 2020](https://www.cbo.gov/system/files/2020-11/56746-MBR.pdf)

In [17]:
# Amount to acheive - reported by congress
print("$", toBillion(df["A02650"].sum()*1000), "Billion")

$ 12811.09 Billion


In [18]:
# Percent each citizen must pay
1609/toBillion(df["A02650"].sum()*1000)

0.12559430930545332

This number mean that most citizens must pay around `15%`, this is an issue though because this means most citizens are pay more than they currently are paying.

In [19]:
expected - paid

agi_stub
1    0.084617
2    0.048989
3    0.036520
4    0.028928
5    0.025455
6    0.016163
dtype: float64

This means according to the tax bracket that poorer individules are not paying their fair share according to the tax bracket.

## Flat Tax Rate Complexity
The issue is tax rate is not this simple. Because the tax brackets are progressive. So we must set a minimum that is taxed and a rate. 

In [20]:
df["rate"] = df.apply(lambda row: row["A06500"] / row["A00100"], axis=1)
PAID_RATE  = df.groupby("agi_stub")["rate"].mean()

In [21]:
def taxSystemRevenue(dataframe, minTaxable, rate):
    df["taxed"] = df.apply(lambda row: max((row["A02650"] - (row["N1"] * minTaxable)), 0) * rate, axis=1)
    taxed       = df.groupby("agi_stub")["taxed"].sum()
    income      = df.groupby("agi_stub")["A02650"].sum()
    rate        = (taxed/income).round(4)
    return rate, taxed.sum()

In [63]:
WEIGHTS = [3, 3, 3, 2, 2, 1]

def scoreTaxSystemRevenue(rate, revenue):
    if toBillion(revenue * 1000) < 1609:
        return 100
    diff  = rate - PAID_RATE
    wdiff = diff * WEIGHTS
    return wdiff.sum()

In [23]:
def simulateTaxSystemRevenue(dataframe, minTaxable, rate):
    rate, revenue = taxSystemRevenue(dataframe, minTaxable, rate)
    return scoreTaxSystemRevenue(rate, revenue)

### Search Grid

In [26]:
MIN_TAXABLE = list(range(0,100, 5))                                 # $0 - $100,000 by 5000s
TAX_RATE    = list(map(lambda x: round(x * 0.01, 2), range(1, 31))) # 0.01% - 0.30% by 0.01

### Perform Search

In [27]:
cols = list(map(lambda x: str(x) + "%", TAX_RATE))
rows = list(map(lambda x: "$" + "{:,}".format(x * 1000), MIN_TAXABLE))
grid = pd.DataFrame(columns=cols, index=rows)

In [64]:
total = len(TAX_RATE) * len(MIN_TAXABLE)
done  = 0

for indexTaxRate in range(len(TAX_RATE)):
    results = []
    for indexMinTax in range(len(MIN_TAXABLE)):
        taxRate = TAX_RATE[indexTaxRate]
        minTax  = MIN_TAXABLE[indexMinTax]
        results.append(simulateTaxSystemRevenue(df, minTax, taxRate))
        done += 1
        print('\r' + str(done) + " of " + str(total), end="\r")
    grid[grid.columns[indexTaxRate]] = results

In [65]:
grid2 = grid.abs()
grid2

,0.01%,0.02%,0.03%,0.04%,0.05%,0.06%,0.07%,0.08%,0.09%,0.1%,...,0.21%,0.22%,0.23%,0.24%,0.25%,0.26%,0.27%,0.28%,0.29%,0.3%
$0,100,100,100,100,100,100,100,100,100,100,...,2.006247,2.146247,2.286247,2.426247,2.566247,2.706247,2.846247,2.986247,3.126247,3.266247
"$5,000",100,100,100,100,100,100,100,100,100,100,...,1.578147,1.697947,1.817747,1.937347,2.056847,2.176147,2.296247,2.415747,2.535047,2.654747
"$10,000",100,100,100,100,100,100,100,100,100,100,...,1.150547,1.250147,1.348947,1.448247,1.547547,1.646847,1.746247,1.845247,1.944647,2.043747
"$15,000",100,100,100,100,100,100,100,100,100,100,...,0.843647,0.928447,1.012747,1.097647,1.182047,1.266747,1.351347,1.435947,1.520747,1.605447
"$20,000",100,100,100,100,100,100,100,100,100,100,...,0.665947,0.742047,0.818547,0.894447,0.970847,1.047047,1.123147,1.199347,1.275447,1.351747
"$25,000",100,100,100,100,100,100,100,100,100,100,...,0.488847,0.556247,0.623947,0.691747,0.759347,0.827047,0.894847,0.962747,1.030147,1.098147
"$30,000",100,100,100,100,100,100,100,100,100,100,...,0.310847,0.370447,0.429447,0.488547,0.548147,0.607147,0.666947,0.725847,0.785147,0.844647
"$35,000",100,100,100,100,100,100,100,100,100,100,...,0.133547,0.184547,0.235447,0.285847,0.336947,0.387747,0.438547,0.489547,0.540047,0.590847
"$40,000",100,100,100,100,100,100,100,100,100,100,...,0.011347,0.056447,0.101147,0.146447,0.191447,0.236247,0.281747,0.326447,0.371547,0.416247
"$45,000",100,100,100,100,100,100,100,100,100,100,...,100.000000,0.039753,0.000747,0.041747,0.081947,0.122947,0.163247,0.203947,0.244447,0.285247


## Deeper Comparison of Best Results
Breakdown the top results to see which system is the best for each tax bracket.

In [52]:
taxRate           = grid.min(axis=0)[12:].reset_index()
minTax            = grid.min(axis=1)[:-2].reset_index()
taxRate.columns   = ["Tax Rate", "Score"]
minTax.columns    = ["Min Taxable", "_"]
breakdown         = taxRate.join(minTax).drop("_", axis=1)

In [73]:
breakdown = grid2.apply(lambda row: row.idxmin(), axis=0).reset_index()
breakdown.columns = ["Tax Rate", "Min Taxable"]

In [74]:
breakdown

,Tax Rate,Min Taxable
0,0.01%,$0
1,0.02%,$0
2,0.03%,$0
3,0.04%,$0
4,0.05%,$0
5,0.06%,$0
6,0.07%,$0
7,0.08%,$0
8,0.09%,$0
9,0.1%,$0


In [75]:
results = [[], [], [], [], [], [], [], [], [], [], [], [], []]
total = len(breakdown)
done  = 0

for index in range(len(breakdown)):
    minTax  = int(breakdown.iloc[index]["Min Taxable"].replace(",", "").replace("$", ""))/1000
    taxRate = float(breakdown.iloc[index]["Tax Rate"].replace("%", ""))
    rate, revenue = taxSystemRevenue(df, minTax, taxRate)
    diff          = rate - PAID_RATE
    results[0].append("$" + str(round(toBillion(revenue * 1000), 0)))
    results[1].append(rate.iloc[0])
    results[2].append(rate.iloc[1])
    results[3].append(rate.iloc[2])
    results[4].append(rate.iloc[3])
    results[5].append(rate.iloc[4])
    results[6].append(rate.iloc[5])
    results[7].append(diff.iloc[0])
    results[8].append(diff.iloc[1])
    results[9].append(diff.iloc[2])
    results[10].append(diff.iloc[3])
    results[11].append(diff.iloc[4])
    results[12].append(diff.iloc[5])
    done += 1
    print('\r' + str(done) + " of " + str(total), end="\r")
breakdown["Revenue"] = results[0]
breakdown["Tax 1"]  = results[1]
breakdown["Tax 2"]  = results[2]
breakdown["Tax 3"]  = results[3]
breakdown["Tax 4"]  = results[4]
breakdown["Tax 5"]  = results[5]
breakdown["Tax 6"]  = results[6]
breakdown["Diff 1"]  = results[7]
breakdown["Diff 2"]  = results[8]
breakdown["Diff 3"]  = results[9]
breakdown["Diff 4"]  = results[10]
breakdown["Diff 5"]  = results[11]
breakdown["Diff 6"]  = results[12]

In [76]:
breakdown

,Tax Rate,Min Taxable,Revenue,Tax 1,Tax 2,Tax 3,Tax 4,Tax 5,Tax 6,Diff 1,Diff 2,Diff 3,Diff 4,Diff 5,Diff 6
0,0.01%,$0,$128.0,0.0100,0.0100,0.0100,0.0100,0.0100,0.0100,-0.002746,-0.034214,-0.055613,-0.068532,-0.100051,-0.178869
1,0.02%,$0,$256.0,0.0200,0.0200,0.0200,0.0200,0.0200,0.0200,0.007254,-0.024214,-0.045613,-0.058532,-0.090051,-0.168869
2,0.03%,$0,$384.0,0.0300,0.0300,0.0300,0.0300,0.0300,0.0300,0.017254,-0.014214,-0.035613,-0.048532,-0.080051,-0.158869
3,0.04%,$0,$512.0,0.0400,0.0400,0.0400,0.0400,0.0400,0.0400,0.027254,-0.004214,-0.025613,-0.038532,-0.070051,-0.148869
4,0.05%,$0,$641.0,0.0500,0.0500,0.0500,0.0500,0.0500,0.0500,0.037254,0.005786,-0.015613,-0.028532,-0.060051,-0.138869
5,0.06%,$0,$769.0,0.0600,0.0600,0.0600,0.0600,0.0600,0.0600,0.047254,0.015786,-0.005613,-0.018532,-0.050051,-0.128869
6,0.07%,$0,$897.0,0.0700,0.0700,0.0700,0.0700,0.0700,0.0700,0.057254,0.025786,0.004387,-0.008532,-0.040051,-0.118869
7,0.08%,$0,$1025.0,0.0800,0.0800,0.0800,0.0800,0.0800,0.0800,0.067254,0.035786,0.014387,0.001468,-0.030051,-0.108869
8,0.09%,$0,$1153.0,0.0900,0.0900,0.0900,0.0900,0.0900,0.0900,0.077254,0.045786,0.024387,0.011468,-0.020051,-0.098869
9,0.1%,$0,$1281.0,0.1000,0.1000,0.1000,0.1000,0.1000,0.1000,0.087254,0.055786,0.034387,0.021468,-0.010051,-0.088869


Based on these numbers to minimize the `$45,000` `22%`